In [ ]:
import pandas as pd
import numpy as np

import skimage
import skimage.io
import skimage.transform
from skimage import exposure

from xgboost import XGBClassifier

from quilt.data.jared import landuse_austin_tx

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def findimg(attop,root,layer) :
    atdir = getattr(attop,root)
    atimg = getattr(atdir,root+layer)
    return atimg

In [ ]:
def prepimage(attop,root): # prepare the image stack
    
    tmpimg = findimg(attop,root,'_B4')
    r=skimage.io.imread(tmpimg(), as_grey=True) # load red
    tmpimg = findimg(attop,root,'_B3')
    g=skimage.io.imread(tmpimg(), as_grey=True) # load green
    tmpimg = findimg(attop,root,'_B2')
    b=skimage.io.imread(tmpimg(), as_grey=True) # load blue
    tmpimg = findimg(attop,root,'_B5')
    ir=skimage.io.imread(tmpimg(), as_grey=True)# load band 5 - near infrared
    
    ndvi = np.true_divide((ir - r), (ir + r)) # ndvi 
    rgb= np.dstack((r,g,b))                   # combine into ordered stack

    tmpimg = findimg(attop,root,'_B8') # load hi-res image
    pan=skimage.io.imread(tmpimg(), as_grey=True)/65535 # load chromatic, normalize to between 0 and 1

    rgb_big=skimage.transform.resize(rgb, output_shape=(pan.shape[0],pan.shape[1],3), order=3, mode='constant', cval=0.0) # resize the rgb composite to match the chromatic
    ndvi_big=skimage.transform.resize(ndvi, output_shape=(pan.shape[0],pan.shape[1]), order=3, mode='constant', cval=0.0) # resize ndvi to match the rgb

    hsv = skimage.color.rgb2hsv(rgb_big) #Convert the rgb to the HSV colorspace
    hsv[...,2]=pan                       #Pan Sharpen
    rgb_pan = skimage.color.hsv2rgb(hsv) #Convert back to rgb space
    
    return rgb_pan, ndvi_big 

In [ ]:
from scipy import ndimage as ndi
from math import floor

metadat = pd.read_json(landuse_austin_tx.metadata.austin_tx_meta())

# load model from file
import pickle
model = pickle.load(open("../output/xboost350_train200_7x7wndvi.sav", "rb"))

In [ ]:
attop = landuse_austin_tx.images.cropped            # use precropped images 
for feature in metadat['features']:
    root=feature['properties']['sceneID']

    #prep images
    rgb, ndvi=prepimage(attop,root) #File name of your imagery
    rgb_dog = rgb - ndi.gaussian_filter(rgb, 15) #Difference of gaussians
    rgb = skimage.img_as_uint(rgb)
    rgb_dog = skimage.img_as_uint(rgb_dog)

    #stack and organize
    stack=np.dstack((rgb, rgb_dog, ndvi))
    depth=stack.shape[2]
    edge=7
    buff=floor(edge/2)
    stack_window=skimage.util.view_as_windows(np.pad(stack,pad_width=((buff,buff), (buff,buff),(0,0)), mode ='minimum'), window_shape=(edge,edge,depth), step=1) 
    print('stack_window.shape ')
    print(stack_window.shape) #7 by 7 moving window around each pixel

    # flatten stack
    stack_flat=stack_window.reshape(-1,edge*edge*depth) 
    print('stack_flat.shape')
    print(stack_flat.shape) #Flatten to rows

    final = model.predict(stack_flat)

    final_img = final.reshape(ndvi.shape)

    plt.figure()
    skimage.io.imshow(final_img)
    plt.show()

    img = np.array(final_img,dtype=np.int8)
    skimage.io.imsave('../output/xgboost_'+root+'.tif',img)

    img = exposure.rescale_intensity(img, out_range='int8')
    skimage.io.imsave('../output/xgboost_'+root+'_rs.tif',img)
